In [13]:
# import all necessary packages & modules
import os
import time
import numpy as np
import pandas as pd
import json
import pyomo.environ as pyomo
from pyomo.opt import ProblemFormat
import matplotlib.pyplot as plt
import networkx as nx
import highspy
from networkx.readwrite import json_graph
from pyomo.opt import SolverStatus, TerminationCondition
from good_model_working import utils


import good_model_working
from good_model_working.reload import deep_reload
from good_model_working import opt_model
from good_model_working import diagnostics
# from good_model_working import Emissions

In [14]:
deep_reload(good_model_working)

# Removing generators for testing
gen_to_remove = [
        #'Fossil Waste', 
        #'Municipal Solid Waste', 
        #'Non-Fossil Waste', 
        'Pumped Storage',
        'Fuel Cell',
        'Landfill Gas', 
        #"Energy Storage", 
        #"Solar PV", 
        #"Onshore Wind", 
        #'New Battery Storage', 
        #'IMPORT', 
        #'Tires',
        'Offshore Wind', 
        'Solar Thermal'
        ]


# import the model data & sets
input_sets = utils.get_sets('/Users/haniftayarani/good_model/Model Input/all_input_sets_sorted.json')
model_sets = utils.filter_sets(input_sets, gen_to_remove)

graph = utils.create_graph('/Users/haniftayarani/good_model/Model Input/all_input_objects.json')

In [15]:
deep_reload(good_model_working)

# MODEL TESTING
'''
    To analyze specific regions, input the appropriate subregion
    to the variable sub_region below
'''

user_input = input('Please select your region(s) for analysis (separated by commas): \nAll\nFlorida\nMISO\nNew England\nPJM\nSSP\nSoutheast\nERCOT\nWECC\n')

subgraph, sub_nodes = utils.get_subgraph(user_input, graph)

constraint_to_deactivate = ['generator', 'solar', 'wind', 'storage']

model_data = {
    'test_nodes': True,
    'test_cons': False,
    'graph': graph, 
    'sets': model_sets, 
    'subgraph_nodes': sub_nodes, 
    'subgraph': subgraph,
    'constraint_deactivation': None
}

In [16]:
# import good_model_working
deep_reload(good_model_working)

#Change this to point to your version of cbc or use another solver
solver_name='appsi_highs'

# run model 
problem = good_model_working.opt_model.Opt_Model(model_data, solver_name)

[+   3.02] Grid built
[+   0.00] Sets built
[+   0.21] Parameters built
Total demand for each region:
Region: ERC_REST, Total Demand: 2667198.447407898
Region: ERC_WEST, Total Demand: 212251.31367483674
Region: FRCC, Total Demand: 1902687.2583417136
Region: MIS_AMSO, Total Demand: 271155.2853672042
Region: MIS_AR, Total Demand: 318652.2256959718
Region: MIS_D_MS, Total Demand: 193407.4098941353
Region: MIS_IA, Total Demand: 185931.05508092826
Region: MIS_IL, Total Demand: 408478.91215887136
Region: MIS_INKY, Total Demand: 812757.4373590639
Region: MIS_LA, Total Demand: 390851.1063049165
Region: MIS_LMI, Total Demand: 832504.9847687938
Region: MIS_MAPP, Total Demand: 74687.97777586698
Region: MIS_MIDA, Total Demand: 255546.0376223612
Region: MIS_MNWI, Total Demand: 798565.5495976977
Region: MIS_MO, Total Demand: 360789.515211062
Region: MIS_WOTA, Total Demand: 282337.800572513
Region: MIS_WUMS, Total Demand: 547558.4370080801
Region: NENGREST, Total Demand: 703578.9056324108
Region: NEN

In [17]:
deep_reload(good_model_working.utils)

# Generate simple model statistics
utils.get_model_statistcs(problem.model)
utils.get_total_generator_count(problem.model)

 total_variables  total_constraints
          312768             319176
Total generators: 3795.0


In [18]:
# Solve the model
problem.solve_model()

# print the objective value
print(f"Total system cost: ${pyomo.value(problem.model.obj_func):,.2f}")

Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
4508 rows, 287545 cols, 304929 nonzeros
4502 rows, 189122 cols, 206462 nonzeros
4502 rows, 189122 cols, 206462 nonzeros
Presolve : Reductions: rows 4502(-314674); columns 189122(-123646); elements 206462(-713665)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
       8815     5.9118854898e+08 Pr: 0(0) 0s
       8815     5.9118854898e+08 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
       8815     5.9119583337e+08 Pr: 2066(0.0079572); Du: 0(8.55183e-07) 0s
      10173     5.9119583351e+08 0s
Model   status      : Optimal
Simplex   iterations: 10173
Objective value     :  5.9118854895e+08
HiGHS run time      :          0.82
To

In [19]:
deep_reload(good_model_working)

# Get model results
results = problem.get_results()

In [24]:
import pickle

# Specify the path to save the pickle file
pickle_file_path = 'results.pickle'

# Save the dictionary as a pickle file
with open(pickle_file_path, 'wb') as f:
    pickle.dump(results, f)
    
nodes_dict = results.get('nodes', {})